In [1]:
from __future__ import division, generators, absolute_import, print_function

import numpy as np
import scipy.stats

# Read Data

In [2]:
from openpyxl import load_workbook
data_dir = '../data/'
wb = load_workbook(data_dir + 'Bishayee Colony Counts 10.27.97-3.8.01.xlsx') 
print(wb.get_sheet_names())
ws = wb.get_sheet_by_name('Sheet1')
print(ws['A1'].value)
rawData = []
for i in range(4, 1366):
    tmp = [ws['D' + str(i)].value, ws['E' + str(i)].value, ws['F' + str(i)].value]
    if isinstance(tmp[0], (int, int)) & isinstance(tmp[1], (int, int)) & isinstance(tmp[2], (int, int)):
        rawData.append(tmp)
print(len(rawData))

['Sheet1']
Bishayee Colony Counts 128 experiments
1361


In [3]:
# Check missing data
sum([len(triple) != 3 for triple in rawData])

0

# Read Mid-probability

In [4]:
output_dir = "../outputs/"

In [5]:
midProb = np.loadtxt(output_dir + "MidProb.txt", delimiter="\t")

In [6]:
midProb.shape

(500, 2)

# Hypothesis III

In [7]:
sortData = rawData
for i in range(len(sortData)):
    sortData[i].sort()

In [8]:
# whether the mid number is the mean of largest and smallest
isMid = [triple[0] + triple[2] == 2 * triple[1] for triple in sortData]
# Total # of obsetvation
obsCount = len(isMid)
# Number of mid number is the mean of largest and smallest
isMidCount = sum(isMid)
# Percentage
isMidPct = isMidCount / obsCount

print("The number of mean containing triples:", isMidCount)
print("The number of triples:", obsCount)
print("The percentage of mean containing triples", isMidPct)

The number of mean containing triples: 263
The number of triples: 1361
The percentage of mean containing triples 0.19324026451138868


* Note that the number of completed entry in the paper is 1343, and the number of mean containing triples is 690.

In [9]:
# rounded mean triple as the estimate of lambda
lamEst = np.rint(np.mean(rawData, axis=1))
# probabiliy that the triple been randomly drawn it would contain its own mean
probs = [midProb[i-1][1] for i in lamEst]

/Users/jorothygong/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [10]:
# expectation and variance of poisson binomial
mu = np.sum(probs)
sigma2 = np.sum(p*(1-p) for p in probs)
print("The mean of poission binomial is", mu)
print("The variance of poission binomial is", sigma2)
print("The sd of poission binomial is", np.sqrt(sigma2))

The mean of poission binomial is 220.313684229
The variance of poission binomial is 180.046603061
The sd of poission binomial is 13.4181445461


In [11]:
# compute statistic for hypothesis 3
stat3 = (isMidCount + 0.5 - mu) / np.sqrt(sigma2)
print("The statistic computed is", stat3)

The statistic computed is 3.2185013079


In [12]:
p_value3 = scipy.stats.norm.pdf(stat3)
print("The p-value is ", p_value3)

The p-value is  0.00224665268132
